In [1]:
import pandas as pd
import requests as re
import os
import pprint
import json

In [2]:
n=200
page = 5

r = re.get(f'https://marketplace.ifood.com.br/v2/merchants?latitude=-19.9727578&longitude=-43.9370045&page={page}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240')

In [3]:
_arr=[]
for _ in range (0,5):
    r = re.get(f'https://marketplace.ifood.com.br/v2/merchants?latitude=-19.9727578&longitude=-43.9370045&page={_}&channel=IFOOD&size={n}&features=&categories=&payment_types=&delivery_fee_from=0&delivery_fee_to=25&delivery_time_from=0&delivery_time_to=240')
    js = json.loads(r.text)
    _arr+= js['merchants']

In [4]:
_arr[0]

{'id': 'a6e1c79f-d289-4783-92b1-73b4ea6e9110',
 'name': 'Coco Bambu Bh Shopping',
 'resources': [{'type': 'LOGO',
   'fileName': 'a6e1c79f-d289-4783-92b1-73b4ea6e9110/202104102132_RVBb_i.png'},
  {'type': 'HEADER',
   'fileName': 'a6e1c79f-d289-4783-92b1-73b4ea6e9110/202103241641_THIf_c.png'}],
 'slug': 'belo-horizonte-mg/coco-bambu-bh-shopping-belvedere',
 'userRating': 4.3348798751831055,
 'priceRange': 'CHEAPEST',
 'mainCategory': {'code': 'FRU', 'name': 'Frutos Do Mar'},
 'features': ['DELIVERY', 'TAKEOUT', 'IMMEDIATE_ORDER'],
 'tags': ['ADDRESS_PREFORM_TYPE',
  'CART::MCHT::TX_GRATIS_CB_100',
  'CRITICOS_CES',
  'CRITICOS_KAS',
  'ESPUMANTE_JUN21',
  'FAMOSOS',
  'FAVORITOS_IFOOD_NOV20',
  'GUIDED_HELP_TYPE',
  'IFOOD_GOURMET',
  'IMPERDIVEIS20_ABR',
  'IMPERDIVEIS70_ABR',
  'INVERNO_MASSAS_JUN21',
  'KEY_ACCOUNT',
  'MELHORES_RESTAURANTES_CHURN_ABR21',
  'MER_ESPETINHO',
  'NOVIDADE_IFOOD_DEZ20 ',
  'PRATOS_ESPETINHOS',
  'PRATOS_FEIJOADA',
  'QTAL_GENERAL',
  'RES_ABR21_FANTA_EG

In [5]:
def get_data(json) -> list:
    
    try:
        _id = json['id']
    except:
        _id=""
    try:
        _name = json['name']
    except:
        _name=""
    try:
        _filename = json['resources'][0]['fileName']
    except:
        _filename=""
    try:
        _cidade = json['slug'].split('/')[0]
    except:
        _cidade=""
    try:
        _price = json['priceRange']
    except:
        _price=""
    try:
        _available = json['available']
    except:
        _available=""
    variaveis = locals()
    variaveis.pop('json')
    return list(variaveis.values())
    

In [6]:
res = list()
for _ in _arr:
    res.append(get_data(_))

In [7]:
df = pd.DataFrame(res,columns=['id','nome','img','localizacao','preco','disponibilidade'])

In [8]:
base = 'https://static-images.ifood.com.br/image/upload/t_thumbnail/logosgde/'
df['img'] = df['img'].apply(lambda x: base + x)

In [9]:
def download_logos(id_: str,  resource: str) -> None:
        pics_path = 'pics'
        if not os.path.exists(pics_path):
            os.mkdir(pics_path)
        
        posfix = resource.split(".")[-1]
        file_name = f"{id_}.{posfix}"
        
        file_path = os.path.join(os.getcwd(), pics_path, file_name)
        
        try:
            response = re.get(url=resource,timeout=10)
        except Exception as e:
            print(e)
        else:  
            with open(file_path, 'wb') as f:
                f.write(response.content)
        

In [10]:
for _,row in df.iterrows():
    download_logos(row.id,row.img)

In [11]:
df.to_csv('lojas-ifood.csv')